In [1]:
import pandas as pd
from IPython.display import clear_output
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [2]:
import time
import random

In [3]:
name_info = pd.read_parquet('./name_info_dtypes.parquet')
name_info.head(2)

,Name,School,Power,Link,Hometown
0,Tiago ASAKAWA,UMBC,-1.0,/swimmer/515075,
1,Niklas Weigelt,UMBC,-1.0,/swimmer/413265,


In [4]:
name_mapping = {
    'Fla Atlantic (M)': 'Florida Atlantic University',
    'Fla Atlantic (W)': 'Florida Atlantic University',
    'Loyola (MD)': 'Loyola University (Maryland)',
    'U.S. Navy': 'U.S. Naval Academy (Navy)',
    'UW-Stevens Point': 'University of Wisconsin-Stevens Point',
    'UWEC': 'University of Wisconsin-Eau Claire',
    'Wash U. MO': 'Washington University (Missouri)',
    'Georgia Tech': 'Georgia Institute of Technology',
    'Virginia MI': 'Virginia Military Institute',
    'Iona Coll': 'Iona University',
    'Wheaton IL': 'Wheaton College (Illinois)',
    'TCNJ': 'College of New Jersey, The',
    'UMSL': 'University of Missouri-St. Louis',
    'Henderson St.': 'Henderson State University',
    'IWU': 'Indiana Wesleyan University',
    'SMU': 'Southern Methodist University',
    'UCSB': 'University of California-Santa Barbara',
    'Missouri St (W)': 'Missouri State University',
    'NC State': 'North Carolina State University',
    'USMMA': 'U.S. Merchant Marine Academy',
    'NDC': 'Notre Dame (Ohio) College',
    'Air Force (W)': 'U.S. Air Force Academy',
    'St. Norbert': 'St Norbert College',
    'W&L': 'Washington and Lee University',
    'UNC Wilmington': 'University of North Carolina, Wilmington',
    'SHIP': 'Shippensburg University',
    'IU Indianapolis': 'Indiana University-Indianapolis',
    'Cal Poly': 'California Polytechnic State University',
    'BSC': 'Birmingham Southern College',
    'Wyoming (M)': 'University of Wyoming',
    'MIT': 'Massachusetts Institute of Technology',
    'Bryant U (W)': 'Bryant University',
    'St. Louis': 'Saint Louis University',
    'UN Omaha': 'University of Nebraska, Omaha',
    'TAMPA': 'University of Tampa',
    'Claremont MS': 'Claremont McKenna-Harvey Mudd-Scripps Colleges',
    'Wheaton MA': 'Wheaton College (Massachusetts)',
    'Evansville (M)': 'University of Evansville',
    'Fla Atlantic (M)': 'Florida Atlantic University',
    'UNE': 'University of New England',
    'Trinity C.': 'Trinity College (Connecticut)',
    'SUNY Geneseo': 'State University of New York at Geneseo',
    'Catholic UA': 'Catholic University',
    'Colorado St.': 'Colorado State University',
    'Frostburg St.': 'Frostburg State University',
    'UNC': 'University of North Carolina, Chapel Hill',
    'F&M': 'Franklin & Marshall College',
    'SUNY Brockport': 'State University of New York at Brockport',
    'Indy': 'University of Indianapolis',
    'Wyoming (W)': 'University of Wyoming',
    'NYU': 'New York University',
    'RIT': 'Rochester Institute of Technology'
}

In [5]:
name_info['School'] = name_info['School'].replace(name_mapping)
name_info[name_info.School=='W&L']

C:\Users\ranadeva\AppData\Local\Temp\ipykernel_7920\3829237642.py:1: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  name_info['School'] = name_info['School'].replace(name_mapping)


,Name,School,Power,Link,Hometown


In [6]:
name_info.Link.value_counts()

Link
                    4048
NoMatch             1061
/swimmer/394628        5
/swimmer/462158        3
/swimmer/403885        3
                    ... 
/swimmer/509068        1
/swimmer/1335673       1
/swimmer/941971        1
/swimmer/1455200       1
/swimmer/167068        1
Name: count, Length: 10130, dtype: Int64

In [7]:
import requests
import csv
from bs4 import BeautifulSoup as bs
import pandas as pd
import time as _time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.action_chains import ActionChains
import re

def getpi(swimmer_name):
    swimmer_name_url = 'https://swimcloud.com/recruiting/rankings/?name='+swimmer_name.replace(' ','+')
    name_url = requests.get(swimmer_name_url, headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36', 'Referer' : 'https://google.com/'})
    name_url.encoding = 'utf-8'
    name_soup = bs(name_url.text, 'html.parser')
    # print(name_soup)
    # print(name_soup)
    # print(name_url.text)
    names=[]
    links=[]
    idxs=[]
    teams=[]
    hometowns=[]
    if (name_soup.find('tbody') == None):
        # print('No tbody')
        return None
    swimmer_list = name_soup.find('tbody').find_all('tr')
    i=0
    for swimmer in swimmer_list:
        if len(name_soup.find_all('td', {'class' : 'u-text-end'})) == 0:
            # print('No idx found')
            break
        name = swimmer.find(class_='u-text-semi')
        name_str = str(name)
        start = name_str.find('>') + 1
        end = name_str.find('</a>')
        # Extract the text between '>' and '<'
        name_raw = name_str[start:end]
        if (name_raw.lower() == swimmer_name.lower() or name_raw.split(' ')[-1].lower() == swimmer_name.lower()):
            names.append(name_raw)
            swimmer_link = name['href'] if name else ''
            links.append(swimmer_link)
            # if (swimmer_name == 'Reese Samuel'):
            #     print(i)
            #     print(name_soup.find_all('td', {'class' : 'u-text-end'})[i])
            if (len(name_soup.find_all('td', {'class' : 'u-text-end'})) == 0):
                return None
            idxs.append(float(name_soup.find_all('td', {'class' : 'u-text-end'})[i].text.strip()))
            team_td = swimmer.find_all('td', class_='u-text-center hidden-xs')[0]
            # Find the <img> tag within the <td>
            img_tag = team_td.find('img')
            if img_tag == None:
                # teams.append('')
                names.pop()
                links.pop()
                idxs.pop()
                # print('None team')
            else:
                # Extract the 'alt' attribute value, which contains the team name
                team_name = img_tag['alt'][:-5]
                teams.append(team_name)
    
    
            # swimmer_name_url = 'https://swimcloud.com'+swimmer_link
            # name_url = requests.get(swimmer_name_url, headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36', 'Referer' : 'https://google.com/'})
            # name_url.encoding = 'utf-8'
            # name_soup2 = bs(name_url.text, 'html.parser')
            # if (name_soup2.find('ul', class_='o-list-inline o-list-inline--dotted') == None):
            #     return None
            # ul = name_soup2.find('ul', class_='o-list-inline o-list-inline--dotted')
            # location_li = ul.find_all('li')[0]
            
            # # Extract and print the location text
            # location = location_li.text.strip()
            # hometowns.append(location)
        i = i + 1
    # print(names)
    # print(links)
    # print(hometowns)
    return [names,links,idxs,teams,hometowns]


In [8]:
all_names = name_info.Name.unique()
j = 0
modified = 0

for j in range(len(all_names)):
    name = all_names[j]
    link_str = name_info.loc[(name_info.Name == name), 'Link']
    found = False
    for sample in link_str:
        if (sample == 'NoMatch' or sample == ''):
            found = True
            
    # print('about to parse ' + name)
    if (j > -1) and found:
    # if (j > -1):
        lists = getpi(name)
        if (lists == None):
            print('No tbody')
            time.sleep(20)
            j = j - 1
        else:
            if len(lists[0]) == 1:
                name_info.loc[(name_info.Name == name), 'Power'] = lists[2][0]
                name_info.loc[(name_info.Name == name), 'Link'] = lists[1][0]
                # name_info.loc[(name_info.Name == name), 'Hometown'] = lists[4][0]
                # print(" Power " + str(lists[2][0]))
                print('Single on ' + str(name))
                modified += 1
            elif len(lists[0]) > 1:
                print(name)
                x_old = name_info[name_info.Name == name]['School'].value_counts()
                x_old = x_old[x_old > 0]
                
                for school in x_old.index:
                    k = 0
                    match = False
                    for school2 in lists[3]:
                        if (school in school2 or school2 in school and school2 != ''):
                            name_info.loc[(name_info.Name == name) & (name_info.School == school), 'Power'] = lists[2][k]
                            name_info.loc[(name_info.Name == name) & (name_info.School == school), 'Link'] = lists[1][k]
                            # name_info.loc[(name_info.Name == name) & (name_info.School == school), 'Hometown'] = lists[4][k]
                            modified += 1
                            match = True
                            # print('Name ' + str(name) + " School " + str(school) + " Power " + str(lists[2][k]))
                            print('Match')
                    if not match:
                        print('No Match')
                        name_info.loc[(name_info.Name == name) & (name_info.School == school), 'Link'] = 'NoMatch'

                        k += 1
            else:
                # print('Empty')
                lastName = name.split(' ')[-1]
                lists = getpi(lastName)
                # print("trying on " + lastName)
                if (lists == None):
                    print('No tbody')
                    time.sleep(20)
                    j = j - 1
                else:
                    # print(len(lists[0]))
                    if len(lists[0]) == 1:
                        name_info.loc[(name_info.Name == name), 'Power'] = lists[2][0]
                        name_info.loc[(name_info.Name == name), 'Link'] = lists[1][0]
                        # name_info.loc[(name_info.Name == name), 'Hometown'] = lists[4][0]
                        # print(" Power " + str(lists[2][0]))
                        print('Single after empty')
                        modified += 1
                # df.loc[(df.Name == name), 'Power'] = -1

    j = j + 1
    # time.sleep(random.uniform(0, 1))
    if (j % 50 == 0):
        clear_output() 
    if (j % 10 == 0):
        print(j)
        print(modified)

200
38
Single after empty
210
39
220
39


KeyboardInterrupt: 

In [ ]:
name_info.Link.value_counts()

In [ ]:
name_info.Name.nunique()

In [ ]:
name_info.shape

In [ ]:
name_mapping = {
    'Fla Atlantic (M)': 'Florida Atlantic University',
    'Fla Atlantic (W)': 'Florida Atlantic University',
    'Loyola (MD)': 'Loyola University (Maryland)',
    'U.S. Navy': 'U.S. Naval Academy (Navy)',
    'UW-Stevens Point': 'University of Wisconsin-Stevens Point',
    'UWEC': 'University of Wisconsin-Eau Claire',
    'Wash U. MO': 'Washington University (Missouri)',
    'Georgia Tech': 'Georgia Institute of Technology',
    'Virginia MI': 'Virginia Military Institute',
    'Iona Coll': 'Iona University',
    'Wheaton IL': 'Wheaton College (Illinois)',
    'TCNJ': 'College of New Jersey, The',
    'UMSL': 'University of Missouri-St. Louis',
    'Henderson St.': 'Henderson State University',
    'IWU': 'Indiana Wesleyan University',
    'SMU': 'Southern Methodist University',
    'UCSB': 'University of California-Santa Barbara',
    'Missouri St (W)': 'Missouri State University',
    'NC State': 'North Carolina State University',
    'USMMA': 'U.S. Merchant Marine Academy',
    'NDC': 'Notre Dame (Ohio) College',
    'Air Force (W)': 'U.S. Air Force Academy',
    'St. Norbert': 'St Norbert College',
    'W&L': 'Washington and Lee University',
    'UNC Wilmington': 'University of North Carolina, Wilmington',
    'SHIP': 'Shippensburg University',
    'IU Indianapolis': 'Indiana University-Indianapolis',
    'Cal Poly': 'California Polytechnic State University',
    'BSC': 'Birmingham Southern College',
    'Wyoming (M)': 'University of Wyoming',
    'MIT': 'Massachusetts Institute of Technology',
    'Bryant U (W)': 'Bryant University',
    'St. Louis': 'Saint Louis University',
    'UN Omaha': 'University of Nebraska, Omaha',
    'TAMPA': 'University of Tampa',
    'Claremont MS': 'Claremont McKenna-Harvey Mudd-Scripps Colleges',
    'Wheaton MA': 'Wheaton College (Massachusetts)',
    'Evansville (M)': 'University of Evansville',
    'Fla Atlantic (M)': 'Florida Atlantic University',
    'UNE': 'University of New England',
    'Trinity C.': 'Trinity College (Connecticut)',
    'SUNY Geneseo': 'State University of New York at Geneseo',
    'Catholic UA': 'Catholic University',
    'Colorado St.': 'Colorado State University',
    'Frostburg St.': 'Frostburg State University',
    'UNC': 'University of North Carolina, Chapel Hill',
    'F&M': 'Franklin & Marshall College',
    'SUNY Brockport': 'State University of New York at Brockport',
    'Indy': 'University of Indianapolis',
    'Wyoming (W)': 'University of Wyoming',
    'NYU': 'New York University',
    'RIT': 'Rochester Institute of Technology'
}

In [ ]:
name_info[name_info.School=='Indy']

In [49]:
name_mapping2 = {
    "St. John's": 'Saint Johns University',
    'St. Cloud St.-M': 'Saint Cloud State University',
    'TCU': 'Texas Christian University',
    'Truman St.': 'Truman State University',
    'Caltech': 'California Institute of Technology',
    'Evansville (W)': 'University of Evansville',
    'FDU-Florham': 'Fairleigh Dickinson University, Madison',
    'Illinois-Chicago': 'University of Illinois at Chicago',
    'Xavier': 'Xavier University',
    'MSU Mankato': 'Minnesota State University-Mankato',
    'Linfield': 'Linfield University',
    'LSU': 'Louisiana State University',
    'UW-L': 'University of Wisconsin-LaCrosse',
    'Keene St.': 'Keene State College',
    'Penn St': 'Pennsylvania State University',
    'SIUC (W)': 'Southern Illinois University',
    'Virginia Tech': 'Virginia Tech',
    'VASSAR': 'Vassar College',
    'R-MC': 'Rainey McCullers School',
    'Concordia Irvine': 'Concordia University Irvine',
    'Trinity U.': 'Trinity University (Texas)',
    'Miami (Ohio)': 'Miami University (Ohio)',
    'Michigan': 'University of Michigan',
    'E. Connecticut': 'Eastern Connecticut State University',
    "Mt St Mary's": "Mount Saint Mary's University (MD)",
    'GW': 'George Washington University',
    'Neb Wesleyan': 'Nebraska Wesleyan University',
    'Northwestern': 'Northwestern University',
    "Nova S'eastern": 'Nova Southeastern University',
    'SUNY Oneonta': 'Oneonta State University',
    'LIU': 'Long Island University',
    'IUP': 'Indiana University of Pennsylvania',
    'L&C': 'Lewis and Clark College',
    'Adams St.': 'Adams State University',
    'CALU-PA': 'Pennsylvania Western-California',
    'Bridgewater St.': 'Bridgewater State University',
    'Wis.- Green Bay': 'University of Wisconsin-Green Bay',
    'CMU': 'Carnegie Mellon University',
    'Virginia': 'University of Virginia',
    'YCP': 'York College (Pennsylvania)',
    'UTPB': 'University of Texas Permian Basin',
    'UNC Asheville': 'University of North Carolina Asheville',
    'UNLV (W)': 'University of Nevada, Las Vegas (UNLV)',
    'Washington St.': 'Washington State University',
    'Connecticut': 'University of Connecticut',
    'Arcadia': 'Arcadia University',
    'UW-Oshkosh': 'University of Wisconsin-Oshkosh',
    'Bryant U (M)': 'Bryant University',
    'W&J': 'Washington & Jefferson College',
    'Northern Ariz': 'Northern Arizona University',
    'WCU': 'West Chester University',
    'WestConn': 'Western Connecticut State University',
    'Notre Dame': 'University of Notre Dame',
    'UNC- Pembroke': 'University of North Carolina-Pembroke',
    'UMary': 'University of Mary',
    'Augustana (M)': 'Augustana College',
    'Army': 'U.S. Military Academy (Army)',
    'Duke': 'Duke University',
    "King's": 'Kings College (Pennsylvania)',
    'Washington': 'Washington College (Maryland)',
    'Sou. Virginia': 'Southern Virginia University',
    'Pacific Lutheran': 'Pacific Lutheran University',
    'Augustana (W)': 'Augustana College',
    "St. Michael's": 'Saint Michaels College',
    'Missouri': 'University of Missouri',
    'Grinnell': 'Grinnell College',
    'Cal Lutheran': 'California Lutheran University',
    'California': 'University of California-Berkeley',
    'GA Southern': 'Georgia Southern University',
    'Hawaii': 'University of Hawaii',
    'Missouri S & T': 'Missouri University of Science & Technology',
    'SUNY New Paltz': 'State University of New York at New Paltz',
    'Alabama': 'University of Alabama',
    'Villanova': 'Villanova University',
    'Monmouth': 'Monmouth University',
    'UC Santa Cruz': 'University of California-Santa Cruz',
    'Montevallo': 'University of Montevallo',
    'CSU East Bay': 'California State University East Bay',
    'SUNY Potsdam': 'State University of New York at Potsdam',
    'Carthage': 'Carthage College',
    'Cal Baptist': 'California Baptist University',
    'Grand Canyon': 'Grand Canyon University',
    'Goucher': 'Goucher College',
    'Pomona-Pitzer': 'Pomona-Pitzer Colleges',
    'Piedmont': 'Piedmont University',
    'Grove City': 'Grove City College',
    'Purdue': 'Purdue University',
    'Illinois Chicago (W)': 'University of Illinois at Chicago',
    'WPUNJ': 'William Paterson University of New Jersey',
    'Hendrix': 'Hendrix College',
    'Rose-Hulman': 'Rose-Hulman Institute of Technology',
    'Virginia Wesleya': 'Virginia Wesleyan University',
    'Hartwick': 'Hartwick College',
    'Auburn': 'Auburn University',
    'Carroll': 'Carroll University (WI)',
    'Penn St. Altoona': 'Pennsylvania State University - Altoona',
    'Assumption': 'Assumption University',
    'McDaniel': 'McDaniel College',
    'Massachusetts': 'University of Massachusetts (Amherst)',
    'Air Force (M)': 'U.S. Air Force Academy',
    'Middlebury': 'Middlebury College',
    'Mines': 'Colorado School of Mines',
    'Minnesota': 'University of Minnesota',
    'Missouri St. (M)': 'Missouri State University',
    'Macalester': 'Macalester College',
    'Wingate': 'Wingate University',
    'Wilmington C.': 'Wilmington College (Ohio',
    'Lynchburg': 'University of Lynchburg',
    'Loy. Marymount': 'Loyola Marymount University',
    'Little Rock': 'University of Arkansas-Little Rock',
    'Nebraska Kearney': 'University of Nebraska at Kearney',
    'Nevada': 'University of Nevada',
    'Lees-McRae': 'Lees-McRae College',
    'Northern Colo': 'University of Northern Colorado',
    'Arizona': 'University of Arizona',
    'Lafayette': 'Lafayette College',
    'Westfield State': 'Westfield State University',
    'Georgia': 'University of Georgia',
    'West Virginia': 'West Virginia University',
    'Ohio St': 'Ohio State University',
    'Old Dominion (W)': 'Old Dominion University',
    'Olivet': 'Olivet Nazarene University',
    'Pacific Oregon': 'Pacific University',
    'John Carroll': 'John Carroll University',
    'Gordon': 'Gordon College (MA)',
    'Behrend': 'Penn State Behrend University',
    'Springfield': 'Springfield College',
    'Davis & Elkins': 'Davis and Elkins College',
    'Texas': 'University of Texas',
    'UNLV (M)': 'University of Nevada, Las Vegas (UNLV)',
    'Clarion': 'Clarion University',
    'CSUB': 'California State University, Bakersfield',
    'St. Francis': 'St Francis College Brooklyn (New York)',
    'Clarkson': 'Clarkson University',
    'Cleveland St': 'Cleveland State University',
    "Saint Peter's": 'Saint Peters University',
    'Texas A&M': 'Texas A&M University',
    'Brown': 'Brown University',
    'Salisbury': 'Salisbury University',
    'St. Thomas': 'University of Saint Thomas (MN)',
    'Emory': 'Emory University',
    'FGCU': 'Florida Gulf Coast University',
    'Seattle U (M)': 'Seattle University',
    'St. Olaf': 'St Olaf College',
    'Chicago': 'University of Chicago',
    'Denver': 'University of Denver',
    'St. Bonaventure': 'St Bonaventure University',
    'Utah': 'University of Utah',
    'Southern Cali': 'University of Southern California',
    'Berry': 'Berry College',
    'SUNY Maritime': 'State Maritime University of New York',
    'Gardner-Webb': 'Gardner-Webb University',
    'South Dakota St': 'South Dakota State University',
    'Campbell': 'Campbell University',
    'Valparaiso (W)': 'Valparaiso University',
    'Florida Tech': 'Florida Tech',
    'Centenary (LA)': 'Centenary College',
    'Transylvania': 'Transylvania University',
    'SUNY Oswego': 'State University of New York at Oswego',
    'Fresno Pacific': 'Fresno Pacific University',
    'Franklin': 'Franklin College',
    'South Dakota': 'University of South Dakota',
    "St. Mary's MD": 'Saint Mary's College of Maryland',
    'St. Francis Pa.': 'Saint Francis University (Pennsylvania)',
    'Oberlin': 'Oberlin College',
    'Northern State': 'Northern State University',
    'North Texas': 'University of North Texas',
    'New Mexico': 'University of New Mexico',
    'New Hampshire': 'University of New Hampshire',
    "St. Joseph's LI": 'Saint Joseph's University (Long Island)',
    "St. Cloud St.-W" 'Saint Cloud State University',
    'Oakland': 'Oakland University',
    'SUNY Fredonia': 'Fredonia State University',
    "St. Kate's" 'St Catherine University',
    'Montclair St.': 'Montclair State University',
    'Tulane': 'Tulane University',
    'Tufts': 'Tufts University',
    'Michigan St': 'Michigan State University',
    'Millikin': 'Millikin University',
    'Mills': 'Mills College',
    'Towson': 'Towson University',
    'Tennessee': 'University of Tennessee',
    'UMBC': 'UMBC',
    'Worcester Poly': 'Worcester Polytechnic Institute',
    'Westminster': 'Westminster College',
    'Sweet Briar': 'Sweet Briar College',
    'Stevens': 'Stevens Institute',
    'Southwestern': 'Southwestern University',
    'Widener': 'Widener University',
    'NJIT': 'New Jersey Institute of Technology',
    'Whitman': 'Whitman College',
    'NMU': 'Northern Michigan University',
    'Wheeling': 'Wheeling University',
    'Staten Island': 'College of Staten Island',
    'Ouachita': 'Ouachita Baptist University',
    'Southwest Minn.': 'Southwest Minnesota State University',
    'Rollins': 'Rollins College',
    'Ramapo': 'Ramapo College of New Jersey',
    'Redlands': 'University of Redlands',
    'Regis': 'Regis College (Massachusetts)',
    'Rensselaer': 'Rensselaer Polytechnic Institute',
    'Rhode Island': 'University of Rhode Island',
    'Rhodes': 'Rhodes College',
    'Rice': 'Rice University',
    'Richmond': 'University of Richmond',
    'Rider': 'Rider University',
    'Ripon': 'Ripon College',
    'Saint Leo': 'Saint Leo University',
    'Occidental': 'Occidental College',
    'Saint Joseph CT': 'University of Saint Joseph',
    'Saginaw Valley': 'Saginaw Valley State University',
    'Rowan': 'Rowan University',
    'Sacred Heart': 'Sacred Heart University',
    'SJCME': 'Saint Josephs College (Maine)',
    'Vermont': 'University of Vermont',
    'Valparaiso (M)': 'Valparaiso University',
    'SUNY Cobleskill': 'State University of New York at Cobleskill',
    'SUNY Cortland': 'State University of New York at Cortland',
    'SUNY Delhi': 'State University of New York at Delhi',
    'Saint Vincent': 'Saint Vincent College',
    'WVWC': 'West Virginia Wesleyan College',
    'Salem (M)': 'Salem University',
    'Salem College': 'Salem College',
    'Oklahoma Baptist': 'Oklahoma Baptist University',
    'Old Dominion (M)': 'Old Dominion University',
    'Wesleyan': 'Wesleyan University',
    'Skidmore': 'Skidmore College',
    'Vanderbilt': 'Vanderbilt University',
    'Ozarks': 'University of the Ozarks',
    'Sioux Falls': 'University of Sioux Falls',
    'Siena': 'Siena College',
    'Penn': 'University of Pennsylvania',
    'Sewanee': 'University of the South (Sewanee)',
    'Seton Hall': 'Seton Hall University',
    'Seattle U (W)': 'Seattle University',
    'Pepperdine': 'Pepperdine University',
    'Pitt-Bradford': 'University of Pittsburgh-Bradford',
    'Merrimack': 'Merrimack College',
    'Principia': 'Principia College',
    'Providence': 'Providence College',
    'Purchase College': 'Purchase College',
    'San Diego St': 'San Diego State University',
    'Queens (NC)': 'Queens University Charlotte',
    'Quincy': 'Quincy University',
    'Miami (FL)': 'University of Miami (Florida)',
    'Messiah': 'Messiah University',
    'Florida': 'University of Florida',
    'Coast Guard': 'U.S. Coast Guard Academy',
    'Biola University': 'Biola University',
    'Greensboro Coll.': 'Greensboro College',
    'Jewell': 'William Jewell College'
}

In [72]:
name_info[(name_info.Link=='NoMatch')].School.value_counts()[290:].head(50)

School
Providence          1
Purchase College    1
San Diego St        1
Queens (NC)         1
Quincy              1
Miami (FL)          1
Messiah             1
Florida             1
Coast Guard         1
Biola University    1
Greensboro Coll.    1
Jewell              1
Cedar Crest         1
Central Conn St     1
George Fox          1
Bard                1
Ball State          1
Kenyon              1
Baldwin Wallace     1
Fordham             1
Knox College        1
Florida St          1
Florida Southern    1
Florida Int'l       1
Cincinnati          1
Buffalo St.         1
Findlay             1
Le Moyne            1
Gustavus            1
Hamilton            1
Iowa                1
Illinois            1
Bucknell            1
Butler              1
Houston             1
Brooklyn            1
Hunter              1
CSU-Pueblo          1
Bowling Green       1
Idaho               1
Bowdoin             1
Carson-Newman       1
Calvin              1
Boston College      1
Immaculata          1
Den

In [71]:
name_info[name_info.School=='Queens (NC)']

,Name,School,Power,Link,Hometown
3495,Skyler Cook-Weeks,Queens (NC),-1.0,/swimmer/345738,
3509,Hendrik Faber,Queens (NC),-1.0,/swimmer/512537,
3535,Bennett Maczka,Queens (NC),-1.0,/swimmer/423465,
3537,Finn Howard,Queens (NC),-1.0,/swimmer/473830,
3544,Luke Erwee,Queens (NC),-1.0,/swimmer/672244,
3549,Mohamed Hegazy,Queens (NC),-1.0,,
3553,Jan Delkeskamp,Queens (NC),-1.0,/swimmer/952843,
3828,Balazs Berecz,Queens (NC),-1.0,/swimmer/508487,
3830,Garrison Johnson,Queens (NC),-1.0,/swimmer/356989,
4161,Kimani Gregory,Queens (NC),-1.0,/swimmer/403057,
